In [3]:
# Import necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.metrics import classification_report 
from sklearn import tree
from z3 import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

class DTModeling:
    def __init__(self, dataset, test_size ):
        self.dataset = dataset
        self.test_size = test_size



    def toFloat(self, str):
        return float(Fraction(str))
  
            
    def getRules(self, dtc):
        rules_list = []
        values_path = []
        values = dtc.tree_.value
        
        def revTraverseTree(tree, node, rules, pathValues):
            try:
                prevnode = tree[2].index(node)           
                leftright = '<='
                pathValues.append(values[prevnode])
            except ValueError:
                # failed, so find it as a right node - if this also causes an exception, something's really f'd up
                prevnode = tree[3].index(node)
                leftright = '>'
                pathValues.append(values[prevnode])

            # now let's get the rule that caused prevnode to -> node
            p1 = tree[0][prevnode]    
            p2 = tree[1][prevnode]    
            #rules.append(str(p1) + ' ' + leftright + ' ' + str(p2))
            rules.append([p1, leftright, p2])

            # if we've not yet reached the top, go up the tree one more step
            if prevnode != 0:
                revTraverseTree(tree, prevnode, rules, pathValues)
    
        # get the nodes which are leaves
        leaves = dtc.tree_.children_left == -1
        leaves = np.arange(0,dtc.tree_.node_count)[leaves]
    
        # build a simpler tree as a nested list: [split feature, split threshold, left node, right node]
        thistree = [dtc.tree_.feature.tolist()]
        thistree.append(dtc.tree_.threshold.tolist())
        thistree.append(dtc.tree_.children_left.tolist())
        thistree.append(dtc.tree_.children_right.tolist())
    
        # get the decision rules for each leaf node & apply them
        for (ind,nod) in enumerate(leaves):
    
            # get the decision rules
            rules = []
            pathValues = []
            revTraverseTree(thistree, nod, rules, pathValues)
    
            pathValues.insert(0, values[nod])      
            pathValues = list(reversed(pathValues))
    
            rules = list(reversed(rules))
    
            rules_list.append(rules)
            values_path.append(pathValues)
    
        return (rules_list, values_path)
    
    def getOptimizedRules(self, dt_m):
        rules = self.getRules(dt_m)[0]
        val_paths = self.getRules(dt_m)[1]
        
        optimized_rules = rules.copy()
        optimized_rules_current_index = 0
        
        for i in optimized_rules[:]:
            optimized_rules[optimized_rules_current_index] = []
            dictionary = {}
            features = []
              
            for item in i:
                if item[0] not in features:
                    features.append(item[0])
                if item[1] == '<=':
                    if ('min_' + str(item[0]) not in dictionary) or (item[2] < dictionary['min_' + str(item[0])]):
                        dictionary['min_' + str(item[0])] = item[2]
                if item[1] == '>':
                    if ('max_' + str(item[0]) not in dictionary) or (item[2] > dictionary['max_' + str(item[0])]):
                        dictionary['max_' + str(item[0])] = item[2]
            
            for j in features:
                if 'min_' + str(j) in dictionary:
                    # Do something
                    current_feature_min = []
                    current_feature_min.append(j)
                    current_feature_min.append('<=')
                    current_feature_min.append(dictionary['min_' + str(j)])
                    optimized_rules[optimized_rules_current_index].append(current_feature_min)
                if 'max_' + str(j) in dictionary:
                    # Do something
                    current_feature_max = []
                    current_feature_max.append(j)
                    current_feature_max.append('>')
                    current_feature_max.append(dictionary['max_' + str(j)])
                    optimized_rules[optimized_rules_current_index].append(current_feature_max)
            
            optimized_rules_current_index += 1
        
        return (optimized_rules, val_paths)
        
    def dtPreprocessing( self ):    
    # features
        self.X = self.dataset.iloc[:,:-1]
        # labels
        self.y= self.dataset.iloc[:,-1] 
    
        # counting input features and output labels
        self.num_inp_features = self.X.shape[1]
        self.num_out_lables = len(np.unique(self.y))
    
        # train-test spilitting
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size = self.test_size, random_state = 0)
        
    def dtModeling( self ):  

        self.dtPreprocessing()
        self.mse_scores = [] # mean_squared_error
        self.mae_scores = [] # mean_absolute_error
        self.r2_scores = [] # r2_score
        self.mape_scores = [] # mean_absolute_percentage_error
#         for i in range(10, 17):
        self.dt_model = DecisionTreeRegressor(max_depth=5, random_state=0)
        self.dt_model.fit(self.X_train, self.y_train)
        y_pred = self.dt_model.predict(self.X_test)
        self.mse_scores.append(mean_squared_error(self.y_test, y_pred))
        self.mae_scores.append(mean_absolute_error(self.y_test, y_pred))
        self.r2_scores.append(r2_score(self.y_test, y_pred))
        self.mape_scores.append(mean_absolute_percentage_error(self.y_test, y_pred))

    def formalZ3Modeling( self):        
        global rules
        self.dtModeling()
        self.s = Solver()
        
        self.z3_input = [Real('input_'+str(i)) for i in range( self.num_inp_features )]
        self.z3_output = Real( 'z3_output' )
        
        dx = [Real('dx_'+str(i)) for i in range( self.num_inp_features )]
        
        for i in range(len(dx)):
            self.s.add(dx[i] == 0)
            #s.add(dx[i] >= 0.5)
            #s.add(dx[i] <= 1)
        
        
        rules = self.getOptimizedRules(self.dt_model)
        
        for i in range ( len(rules[0] ) ):
            and_rules = []
            
            for j in range ( len(rules[0][i] ) ):
                if rules[0][i][j][1] == '<=':        
                    and_rules.append( self.z3_input[ rules[0][i][j][0] ] + dx[ rules[0][i][j][0] ] <= 
                                               rules[0][i][j][2] )
                elif rules[0][i][j][1] == '>':        
                    and_rules.append( self.z3_input[ rules[0][i][j][0] ] + dx[ rules[0][i][j][0] ] > rules[0][i][j][2] )
            length = len(rules[1][i])
            #label = np.argmax( np. asarray(rules[1][i][length - 1]) )
            label = rules[1][i][len(rules[1][i]) - 1][0]
            self.s.add( Implies( And( and_rules ), self.z3_output ==  int(label) ) )
        
#         for i in range( self.X.shape[1] ):
#             self.s.add( z3_input[i] == self.input_val[i])
        
#         if self.s.check() == unsat:
#             return -1
#         return self.toFloat(str(self.s.model()[z3_output])), self.dt_model.predict([self.input_val.tolist()])
#         return self.s
    


dataset = pd.read_csv('output.csv')
dataset = dataset.drop(dataset.columns[0], axis=1)
# dataset = dataset.drop(dataset.columns[0], axis=1)
# dataset = dataset.drop(dataset.columns[-2], axis=1)

dataset.iloc[:, -1] *= 100
dataset.iloc[:, 0] *= 100
dataset.iloc[:, -2] *= 100


dataset = dataset.iloc[: , [0, -2, -1]].copy()
dataset.head()

X = dataset.iloc[:, :-1]
y = dataset.iloc[:, -1]



# y= preprocessing.LabelEncoder().fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (mean=0 and variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the logistic regression model
model = DecisionTreeRegressor(random_state=0)
model.fit(X_train, y_train)

# Predictions on the test set
y_pred = model.predict(X)


#---------------------------------------------------------------
# print(y_pred)

# for i in range(len(dataset)):
#     input_val = dataset.iloc[i, :-1].values
# #     print(input_val)
# #     print(type(input_val))
#     dt_model1 = DTModeling(dataset, 0.2)
#     constraints,pred,solver = dt_model1.formalZ3Modeling(input_val) 
#     print(y[i],pred, constraints)
    
    

dt_model1 = DTModeling(dataset , 0.2)

dt_model1.formalZ3Modeling() 


# #---------------------------------------------#
# #---------------------------------------------#
# dt_model1.s.push()
# temp_arr = np.array([0.99281075, 0.95845847])

# for i in range( dt_model1.X.shape[1] ):
#     dt_model1.s.add( dt_model1.z3_input[i] == temp_arr[i])

# if dt_model1.s.check() == unsat:
#     print("No result")
# else:                    
#     print(dt_model1.toFloat(str(dt_model1.s.model()[dt_model1.z3_output])), dt_model1.dt_model.predict([temp_arr.tolist()]))
# dt_model1.s.pop()
# #---------------------------------------------#



# #---------------------------------------------#
# #---------------------------------------------#
# dt_model1.s.push()    
# temp_arr = np.array([200, 180])

# for i in range( dt_model1.X.shape[1] ):
#     dt_model1.s.add( dt_model1.z3_input[i] == temp_arr[i])

# if dt_model1.s.check() == unsat:
#     print("No result")
# else:                    
#     print(dt_model1.toFloat(str(dt_model1.s.model()[dt_model1.z3_output])), dt_model1.dt_model.predict([temp_arr.tolist()]))
# dt_model1.s.pop()
# #---------------------------------------------#


# #---------------------------------------------#
# #---------------------------------------------#
# dt_model1.s.push()    
# temp_arr = np.array([120, 100])

# for i in range( dt_model1.X.shape[1] ):
#     dt_model1.s.add( dt_model1.z3_input[i] == temp_arr[i])

# if dt_model1.s.check() == unsat:
#     print("No result")
# else:                    
#     print(dt_model1.toFloat(str(dt_model1.s.model()[dt_model1.z3_output])), dt_model1.dt_model.predict([temp_arr.tolist()]))
# dt_model1.s.pop()
# #---------------------------------------------#



initial_balances = [100, 200, 150, 145, 105]
ending_balances = [120, 180, 150, 120, 130]

# sender_counts = [2, 1, 2, 1, 1]
# receiver_counts = [1, 2, 2, 1, 1]

# initial_balances = [30, 17, 25, 18]
# ending_balances = [36, 25, 15, 14]

# sender_counts = [1, 1, 1, 2]
# receiver_counts = [1, 2, 1,1]

# initial_balances = [30, 17, 25]
# ending_balances = [36, 25, 11]

# sender_counts = [2, 1, 1]
# receiver_counts = [1, 2, 1]

# Define variables
n_users = len(initial_balances)

user_idxs = list(range(0, n_users))

tactical_channel = 10


transactions = [ [[Int(f"txs_{i}_to_{j}_idx{k}") for k in range(tactical_channel)] for j in range(n_users) if j!=i] for i in range(n_users)]
amounts = [ [[Int(f"coin_{i}_to_{j}_idx{k}") for k in range(tactical_channel)] for j in range(n_users) if j!=i] for i in range(n_users)]

# # print(len(transactions))
# print(transactions)

# # print(len(amounts))
# print(amounts)

sends = [[Int(f"sender_{i}_tx{k}") for k in range(tactical_channel)] for i in range(n_users)]
receives = [[Int(f"receiver_{i}_tx{k}") for k in range(tactical_channel)] for i in range(n_users)]

balances = [[Int(f"balance_{i}_tx{k}") for k in range(tactical_channel+1)] for i in range(n_users)]


# Create solver instance
# solver = Solver()

for i in range(n_users):
    for k in range(tactical_channel):
        dt_model1.s.add(sends[i][k] >= 0)
        
for i in range(n_users):
    for k in range(tactical_channel):
        dt_model1.s.add(receives[i][k] >= 0)

# for i in range(n_users): 
#     dt_model1.s.add(Sum([If(value != 0, 1, 0) for value in sends[i]]) == sender_counts[i])
#     dt_model1.s.add(Sum([If(value != 0, 1, 0) for value in receives[i]]) == receiver_counts[i])



for i in range(n_users):
    dt_model1.s.add(balances[i][0] == initial_balances[i])
    dt_model1.s.add(balances[i][-1] == ending_balances[i])


# for j in range(tactical_channel+1):
#     total_blnc_tx_j = [user_blnc_tx_j[j] for user_blnc_tx_j in balances]
#     solver.add(sum(total_blnc_tx_j) == sum(initial_balances))


for i in range(n_users):
    for j in range(1, tactical_channel+1):
        dt_model1.s.add(balances[i][j] == (balances[i][j-1]-sends[i][j-1]+receives[i][j-1]))

for i in range(n_users):
    for k in range(tactical_channel):
        dt_model1.s.add(sends[i][k] <= balances[i][k])


        
for k in range(tactical_channel):
    for i in range(n_users):
        others = [x for x in user_idxs if x != i]
        conditions = []
        for m in others:
            conditions.append(receives[m][k]==sends[i][k])
        dt_model1.s.add(Or(conditions))
        
for i in range(tactical_channel):
    temp_senders = []
    temp_receivers = []
    for k in range(n_users):
        temp_senders.append(sends[k][i])
        temp_receivers.append(receives[k][i])
    dt_model1.s.add(Sum([If(value != 0, 1, 0) for value in temp_senders]) == 1)
    dt_model1.s.add(Sum([If(value != 0, 1, 0) for value in temp_receivers]) == 1)

    
# New constraints for restricting the predictions    
for i in range(n_users):
    
    middle = 0
    #---------------------------------------------#
    #---------------------------------------------#
    dt_model1.s.push()    
    temp_arr = np.array([initial_balances[i], ending_balances[i]])

    for i in range( dt_model1.X.shape[1] ):
        dt_model1.s.add( dt_model1.z3_input[i] == temp_arr[i])

    if dt_model1.s.check() == unsat:
        print("No result")
    else:
        middle = dt_model1.toFloat(str(dt_model1.s.model()[dt_model1.z3_output]))
        print(dt_model1.toFloat(str(dt_model1.s.model()[dt_model1.z3_output])), dt_model1.dt_model.predict([temp_arr.tolist()]))
    dt_model1.s.pop()
    #---------------------------------------------#
    
    for k in range(tactical_channel):
        constraint = Implies(sends[i][k] > 0, And(sends[i][k] >= int(middle*0.5), sends[i][k] <= int(middle*1.5)))

        # Add the constraint to the solver
        dt_model1.s.add(constraint)
#         dt_model1.s.add(int(middle*0.5)<=sends[i][k]) 
#         dt_model1.s.add(sends[i][k] <= int(middle*1.5))







# print(len(sends))
# print("\nSends",sends)

# print("\nReceives",receives)

# print("\nBalances",balances)


print("Initial Balances",initial_balances)
print("Ending Balances",ending_balances)

# print("Sender count",sender_counts)
# print("Receiver count",receiver_counts)

txs = []

# Solve the problem
if dt_model1.s.check() == sat:
    model = dt_model1.s.model()


#     for i in range(n_users):
#         print("\n")
#         for k in range(tactical_channel+1):
#             if k==0:
#                 print(f"Banalance of user {i} at the begining", model[balances[i][k]])
#             else:
#                 print(f"Banalance of user {i} after TX{k}", model[balances[i][k]])
    
    
    
    for k in range(tactical_channel):
#         print("\n")
        tempSend = None
        tempRcv = None
        amount = None

        for i in range(n_users):
#             if (int(str(model[sends[i][k]]) != 0):
#                    print(f"\nUser {i} sends on TX{k+1}",model[sends[i][k]])
#             if int(str(model[receives[i][k]])) != 0:
#                    print(f"User {i} receives on TX{k+1}",model[receives[i][k]])
                
            temp1 =int(str(model[sends[i][k]]))
            if temp1 !=0:
#                 print(f"User {i} sends on TX{k+1}",model[sends[i][k]])
                tempSend = i
                amount = temp1
                
            
            temp2 =int(str(model[receives[i][k]]))
            if temp2 !=0:
#                 print(f"User {i} receives on TX{k+1}",model[receives[i][k]])
                tempRcv = i
            
            
        txs.append([tempSend, tempRcv, amount])
else:
    print("No solution found.")

for i in range(len(txs)):
    print(f"\nUser{txs[i][0]} sends {txs[i][2]} coin(s) to User{txs[i][1]} on TX{i}")
    balns = []
    for j in range(n_users):
        balns.append(int(str(model[balances[j][i+1]])))
    print(balns)
    
for i in range(tactical_channel):
    total = 0
    for j in range(n_users):
        total+=int(str(model[balances[j][i]]))
#     print(total)

97.0 [97.93334042]
165.0 [165.1399911]
165.0 [165.1399911]
165.0 [165.1399911]
97.0 [97.93334042]
Initial Balances [100, 200, 150, 145, 105]
Ending Balances [120, 180, 150, 120, 130]

User0 sends 1 coin(s) to User4 on TX0
[99, 200, 150, 145, 106]

User1 sends 82 coin(s) to User3 on TX1
[99, 118, 150, 227, 106]

User3 sends 104 coin(s) to User0 on TX2
[203, 118, 150, 123, 106]

User1 sends 82 coin(s) to User0 on TX3
[285, 36, 150, 123, 106]

User3 sends 3 coin(s) to User2 on TX4
[285, 36, 153, 120, 106]

User2 sends 1 coin(s) to User4 on TX5
[285, 36, 152, 120, 107]

User0 sends 21 coin(s) to User4 on TX6
[264, 36, 152, 120, 128]

User0 sends 144 coin(s) to User1 on TX7
[120, 180, 152, 120, 128]

User2 sends 1 coin(s) to User4 on TX8
[120, 180, 151, 120, 129]

User2 sends 1 coin(s) to User4 on TX9
[120, 180, 150, 120, 130]


C:\Users\akhalil\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
C:\Users\akhalil\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhalil\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhalil\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\akhalil\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn